In [9]:
# Created by Ian Cox | 2/27/2023
# AHEAD | March Madness 2023 Bracket Challenge
# Tournament Prediction |SKLearn classifiers with gridsearch param tuning
# Pick winners based on tournament play history,  regular season stats up to the tournament, and seed

# change working dir
import os
os.chdir('C:\\Users\\IanCox\\OneDrive - AHEAD\\Documents\\python\\march_madness\\march-machine-learning-mania-2023')

# load the ml ready data
import pandas as pd
data = pd.read_csv('tourney_ml_input.csv')

# import ml lib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

In [7]:
# Split the data into training and test sets
train_data = data[data["Season"] < 2017]
test_data = data[data["Season"] >= 2017]

In [11]:
# Split the features and target variable into X and y keeping the temporal association intact
# Select the features and target variable
features = ["RS Point Total", "RS Avg Points", "OPP RS Point Total", "OPP RS Avg Points", "SeedDiff", "TeamID", "OpponentID"]
target = "outcome"

X_train = train_data[features]
y_train = train_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [17]:
# Define models to evaluate with parameter tuning
models = {
    'Logistic Regression': GridSearchCV(LogisticRegression(),
                                        {'C': [0.1, 1.0, 10.0],
                                         'penalty': ['l1', 'l2']}),
    'Decision Tree': GridSearchCV(DecisionTreeClassifier(),
                                  {'max_depth': [3, 5, 7],
                                   'min_samples_split': [2, 5, 10]}),
    'Random Forest': GridSearchCV(RandomForestClassifier(),
                                  {'n_estimators': [50, 100, 150],
                                   'max_depth': [3, 5, 7],
                                   'min_samples_split': [2, 5, 10]}),
    'Gradient Boosting': GridSearchCV(GradientBoostingClassifier(),
                                       {'n_estimators': [50, 100, 150],
                                        'learning_rate': [0.01, 0.1, 1.0],
                                        'max_depth': [3, 5, 7]}),
    'K-Nearest Neighbors': GridSearchCV(KNeighborsClassifier(),
                                         {'n_neighbors': [3, 5, 7],
                                          'weights': ['uniform', 'distance'],
                                          'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']})
}

In [20]:
def evaluate_models(X_train, y_train, X_test, y_test):
    # Evaluate each model and store the results in a dictionary
    results = {}
    for name, model in models.items():
        # Fit the model
        model.fit(X_train, y_train)

        # Make predictions on test data
        y_pred = model.predict(X_test)

        # Calculate accuracy score
        accuracy = accuracy_score(y_test, y_pred)

        # Store the accuracy score for each model in a dictionary
        results[name] = accuracy
    
    # Convert the results dictionary into a pandas dataframe and sort by accuracy score in descending order
    results_df = pd.DataFrame.from_dict(results, orient='index', columns=['Accuracy Score']).sort_values(by='Accuracy Score', ascending=False)
    
    return results_df

In [21]:
evaluate_models(X_train, y_train, X_test, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueError(
Valu

,Accuracy Score
Random Forest,0.684451
Logistic Regression,0.679878
Decision Tree,0.672256
Gradient Boosting,0.669207
K-Nearest Neighbors,0.612805
